# Object-Oriented Programming
OOP is a powerful programming paradigm that allows us to organize code into objects, which are self-contained units encapsulating data and behavior. This paradigm is widely used in software development and is especially beneficial when dealing with complex systems or simulations. By adopting an object-oriented approach, we can create modular, reusable, and maintainable code.

To illustrate the principles of OOP, we will focus on a practical example: pricing a European Call Option. A Call Option gives the holder the right, but not the obligation, to buy an underlying asset at a predetermined price (the strike price) within a specific time period (until the option's expiration date). The pricing of options is a fundamental concept in finance and is extensively used in derivatives trading.

By developing an object to price a European Call Option, we will demonstrate how to structure code using classes, define attributes and methods, and implement key functionalities. This notebook will guide you step-by-step through the process, allowing you to grasp the core concepts of OOP.

## Equity Class
In this section, we will focus on developing a fundamental building block for pricing European Call Options: the Equity class. Before we can price options, we need to understand the underlying asset on which the options are based. An equity represents ownership in a company or an asset, such as stocks or shares. By creating an Equity class, we establish a foundation for modeling and analyzing the behavior of the underlying asset.

In [1]:
class Equity:
    """
        An equity represents ownership in a company or an asset, such as stocks or shares.
    """
    def __init__(self, s, n):
        """
        Parameters:
            s (float): the share price
            n (int):   the number of shares
        """
        self.s = s
        self.n = n
    
    @property
    def theoretical_value(self):
        return self.s
    
    @property
    def market_value(self):
        return self.theoretical_value*self.n
    
    @property
    def delta(self):
        return 1
    
    @property
    def total_delta(self):
        return self.delta*self.n*self.s
    
    def update_share_price(self, new_price):
        self.s = new_price

In [2]:
apple = Equity(s=175, n=10)
print('Market Value: ${:2}'.format(apple.market_value))
apple.update_share_price(185)
print('Market Value: ${:2}'.format(apple.market_value))


Market Value: $1750
Market Value: $1850


## Option class

In [3]:
import numpy as np
import scipy.stats as sp

class EuCall(Equity):
    def __init__(self,t, s, k, r, q, v,n):
        """
        t = maturity
        s = spot price (underlying)
        k = strike
        r = interest rate
        q = div yield
        v = volatility
        n = nbr of options
        Example : EuCall(5,100,100,0.03,0.01,0.4,100)
        """
        self.s = s
        self.t = t
        self.k = k
        self.r = r
        self.q = q
        self.v = v
        self.n = n
    
    def __str__(self):
        return str(self.n)+"x"+" EUCALL Strike=" + str(self.k)
    
    @property
    def theoretical_value(self):
        if self.t <= 0:
            return max(self.s - self.k, 0)
        else:
            d1 = (1/(self.v*np.sqrt(self.t))) * (np.log(self.s/self.k)+ (self.r-self.q+0.5*self.v**2)*self.t)
            d2 = d1 - self.v*np.sqrt(self.t)
            return self.s*np.exp(-self.q*self.t)*sp.norm().cdf(d1)-sp.norm().cdf(d2)*self.k*np.exp(-self.r*self.t)
        
    @property
    def delta(self):
        s0 = self.s
        s_plus = self.s*1.001
        s_minus = self.s*(0.999)
        self.s = s_plus
        theovalue_plus = self.theoretical_value
        self.s = s_minus
        theovalue_minus = self.theoretical_value
        self.s = s0
        return (theovalue_plus - theovalue_minus)/(s_plus-s_minus)

In [4]:
a = EuCall(1,100,100,0.03,0.01,0.4,1)
print('Delta:        {}'.format(round(a.delta, 4)))
print('Market Value: {}'.format(round(a.market_value, 4)))

Delta:        0.5927
Market Value: 16.5382
